# Двухфакторный дисперсионный анализ (Two-Way ANOVA)
Данный метод анализа данных применяется в тех случаях, когда мы хотим узнать, как различные свойства данных влияют на данные в целом. В частности, как два свойства влияют друг на друга.
Для того, чтобы провести двухфакторный дисперсионный анализ, необходимо ввести следующие понятия:
- SST (Sum square total) - Общая сумма квадратов;
- SSB_a (Sum square between) - Межгрупповая сумма квадратов групп по свойству А;
- SSB_b (Sum square between) - Межгрупповая сумма квадратов групп по свойству B;
- SSb_a*b - Межгрупповая сумма квадратов взаимодействия различных групп;
- SSW (Sum square within) - Внутригрупповая сумма квадратов.

Тогда:
SST = SSW + SSB_a + SSB_b + SSB_a*b
SSB_a*b можно найти, зная все остальные слагаемые и общую сумму квадратов.

Ниже по шагам вручную будет представлен алгоритм применения двухфакторного дисперсионного анализа.
Также предлагаю плейлист коротких видео с объяснением двухфакторного анализа - его рассчетами и интерпретацией результатов: 

Первым делом импортируем все необходимые библиотеки и создадим анализируемый датасет. Будем применять алгоритм на примере анализа данных об успеваемости студентов в зависимости от их пола и факультета.

In [49]:
import math
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

df = pd.DataFrame(
    {
        "Sex": ["Male", "Male", "Male", "Female", "Female", "Female", "Male", "Male", "Male", "Female", "Female", "Female", "Male", "Male", "Male", "Female", "Female", "Female"],
        "Department": ["Math", "Math", "Math", "Math", "Math", "Math", "Biology", "Biology", "Biology", "Biology", "Biology", "Biology", "Politics", "Politics", "Politics", "Politics", "Politics", "Politics"],
        "Score": [9, 8, 9, 5, 4, 6, 6, 5, 6, 10, 9, 10, 5, 7, 6, 6, 7, 7]  
    }
)

df

,Sex,Department,Score
0,Male,Math,9
1,Male,Math,8
2,Male,Math,9
3,Female,Math,5
4,Female,Math,4
5,Female,Math,6
6,Male,Biology,6
7,Male,Biology,5
8,Male,Biology,6
9,Female,Biology,10


Теперь посчитаем все необходимые средние значения для всех возможных групп.

In [50]:
# Общее среднее
overall_average = np.average(df["Score"])                                     

# Средние для групп по свойству "Пол"
male_average   = np.average(df.loc[df["Sex"] == "Male"]["Score"])            
female_average = np.average(df.loc[df["Sex"] == "Female"]["Score"])          

# Средние для групп по свойству "Факультет"
math_average     = np.average(df.loc[df["Department"] == "Math"]["Score"])     
biology_average  = np.average(df.loc[df["Department"] == "Biology"]["Score"])  
politics_average = np.average(df.loc[df["Department"] == "Politics"]["Score"]) 

# Средние для каждых групп по всем свойствам
male_math_average       = np.average(df.loc[(df["Sex"] == "Male") & (df["Department"] == "Math")]["Score"])
female_math_average     = np.average(df.loc[(df["Sex"] == "Female") & (df["Department"] == "Math")]["Score"])
male_biology_average    = np.average(df.loc[(df["Sex"] == "Male") & (df["Department"] == "Biology")]["Score"])
female_biology_average  = np.average(df.loc[(df["Sex"] == "Female") & (df["Department"] == "Biology")]["Score"])
male_politics_average   = np.average(df.loc[(df["Sex"] == "Male") & (df["Department"] == "Politics")]["Score"])
female_politics_average = np.average(df.loc[(df["Sex"] == "Female") & (df["Department"] == "Politics")]["Score"])

overall_average

6.944444444444445

Далее посчитаем все необходимые суммы:

In [51]:
### КАКАЯ ТО ОШИБКА В РАСЧЕТАХ
# Общая сумма квадратов
SST = np.sum([math.pow(score - overall_average, 2) for score in df["Score"]])

# Внутригрупповая сумма квадратов
SSW = np.sum([math.pow(score - male_math_average, 2) for score in df.loc[(df["Sex"] == "Male") & (df["Department"] == "Math")]["Score"]]) \
+ np.sum([math.pow(score - female_math_average, 2) for score in df.loc[(df["Sex"] == "Female") & (df["Department"] == "Math")]["Score"]]) \
+ np.sum([math.pow(score - male_biology_average, 2) for score in df.loc[(df["Sex"] == "Male") & (df["Department"] == "Biology")]["Score"]]) \
+ np.sum([math.pow(score - female_biology_average, 2) for score in df.loc[(df["Sex"] == "Female") & (df["Department"] == "Biology")]["Score"]]) \
+ np.sum([math.pow(score - male_politics_average, 2) for score in df.loc[(df["Sex"] == "Male") & (df["Department"] == "Politics")]["Score"]]) \
+ np.sum([math.pow(score - female_politics_average, 2) for score in df.loc[(df["Sex"] == "Female") & (df["Department"] == "Politics")]["Score"]])

# Межгрупповая сумма по свойству "Пол"
SSB_a = np.sum([math.pow(score - male_average, 2) for score in df.loc[df["Sex"] == "Male"]["Score"]]) \
+ np.sum([math.pow(score - female_average, 2) for score in df.loc[df["Sex"] == "Female"]["Score"]])
print(df["Score"], SST, SSW, SSB_a)

0      9
1      8
2      9
3      5
4      4
5      6
6      6
7      5
8      6
9     10
10     9
11    10
12     5
13     7
14     6
15     6
16     7
17     7
Name: Score, dtype: int64 56.94444444444445 6.666666666666667 56.44444444444444


Автоматическое применение Two-Way ANOVA с помощью statsmodels:

In [52]:
model = ols('Score ~ C(Sex) + C(Department) + C(Sex):C(Department)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Sex),0.500000,1.0,0.9,0.361497
C(Department),5.444444,2.0,4.9,0.027819
C(Sex):C(Department),44.333333,2.0,39.9,0.000005
Residual,6.666667,12.0,NaN,NaN
